Import packages

In [1]:
import tensorflow as tf
from tensorflow.keras import layers as tfl
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.initializers import glorot_uniform
from keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
import os
cwd = os.getcwd()
directory = cwd.replace("\\", "/")[:-9]
import gradio as gr

Define f1_score

In [2]:
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

Prepare the test dataset

In [3]:
BATCH_SIZE = 40
IMG_SIZE = (224, 224)


test_dataset = image_dataset_from_directory(directory+"/images/testing/",
                                            shuffle=True,
                                            batch_size=BATCH_SIZE,
                                            image_size=IMG_SIZE,
                                            seed=42)

class_names = test_dataset.class_names

print(f'The class names are: {class_names}')

Found 597 files belonging to 2 classes.
The class names are: ['flip', 'notflip']


Define custom object BottleneckBlock before loading the model

In [4]:
class BottleneckBlock(tf.keras.layers.Layer):
    """
    Bottleneck block from MobileNetV2
    
    Arguments:
    f -- shape of the convolutional window
    nb_depth -- number of filters to expand to for the depthwise convolution
    nb_project -- number of filters for the projection
    
    """
    def __init__(self, f, nb_depth, nb_proj, **kwargs):
        super(BottleneckBlock, self).__init__(**kwargs)
        self.expconv = tfl.Conv2D(filters = nb_depth, 
                                  kernel_size = (1,1), 
                                  strides = (1,1), 
                                  padding = 'same', 
                                  kernel_initializer = glorot_uniform(seed=42),
                                  name = 'expconv'
                                  )
        self.expbatch = tfl.BatchNormalization(axis = 3, name = 'expbatch')
        self.depthconv = tfl.DepthwiseConv2D(kernel_size = (f,f), 
                                             strides = (1,1), 
                                             padding = 'same', 
                                             kernel_initializer = glorot_uniform(seed=42),
                                             name = 'depthconv'
                                             )
        self.depthbatch = tfl.BatchNormalization(axis = 3, name = 'depthbatch')
        self.projconv = tfl.Conv2D(filters = nb_proj, 
                                   kernel_size = (1,1), 
                                   strides = (1,1), 
                                   padding = 'same', 
                                   kernel_initializer = glorot_uniform(seed=42),
                                   name = 'projconv'
                                   )
        self.projbatch = tfl.BatchNormalization(axis = 3, name = 'projbatch')
        self.shortbatch = tfl.BatchNormalization(axis = 3, name = 'shortbatch')
        self.nb_depth = nb_depth
        self.nb_proj = nb_proj
        self.f = f
    
    def call(self, X):
        X_shortcut = X # Saved for the shortcut connection
    
        # Expansion Part
        X = self.expconv(X)
        X = self.expbatch(X)
        X = tfl.Activation('relu')(X)
        
        # Depthwise Convolution Part
        X = self.depthconv(X)
        X = self.depthbatch(X)
        X = tfl.Activation('relu')(X)
        
        # Projection Part
        X = self.projconv(X)
        X = self.projbatch(X)
        
        # Shortcut Connection
        if self.nb_proj == X_shortcut.shape[3]:
            X_shortcut = self.shortbatch(X_shortcut)
            X = tfl.add([X, X_shortcut])
        X = tfl.Activation('relu')(X)
    
        return X
    
    def get_config(self):
        config = super(BottleneckBlock, self).get_config()
        config.update({"f": self.f, "nb_depth": self.nb_depth, "nb_proj": self.nb_proj})
        return config

Load the model with its custom objects

In [5]:
custom_objects = {"BottleneckBlock": BottleneckBlock, "f1_score": f1_score}
loaded_model = tf.keras.models.load_model('./checkpoints/custom_model.h5', custom_objects=custom_objects)

Pick some examples

In [6]:
examples = []
for images, labels in test_dataset.take(1):
    for i in range(BATCH_SIZE):
        examples.append(images[i].numpy().astype("uint8"))

Define a function that gradio app can use

In [7]:
def classify_image(example):
    # Returns a dictionary whose keys are the labels and values their predicted probabilities
    prediction = loaded_model.predict(np.reshape(example, (1,) + example.shape))[0][0]
    probs = {'not flip':float(prediction), 'flip':float(1-prediction)}
    return probs

Generate Gradio Interface

In [8]:
interface = gr.Interface(fn=classify_image,
                         inputs=gr.Image(shape=(224, 224)),
                         outputs=gr.Label(num_top_classes=2),
                         description = "Click on one of the dots below Examples to generate an example",
                         title = "Flip detector",
                         examples=examples)

In [9]:
interface.launch(share = True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://f541056b17bb9c0211.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
